In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
## IMPORTS
import openai
import pandas as pd
import numpy as np
import json
from CONFIG import PRODUCT_PROMPT, ISSUE_TYPE_PROMPT, SERVICES_PROMPT, RELATIONSHIP_PROMPT
import os 
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

# Load environment variables from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set OpenAI API key
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [15]:
## Product Extraction
def extract(text,prompt):

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ],
        temperature=0,
        top_p=0.95
    )

    return response.choices[0].message.content

In [16]:
data_name = "twcs_structured_UniqueCount-10_time-20250608-1251"

In [17]:
data = pd.read_excel(f'..\\..\\data\processed\sample\\{data_name}.xlsx')

In [18]:
data['Product'] = np.nan
data['Issue Type'] = np.nan
data['Services'] = np.nan

In [19]:
for i in tqdm(range(len(data))):
    if pd.isna(data['Product'][i]):
        input_text = data['structured_conversations'][i]
        product = extract(input_text, PRODUCT_PROMPT)
        data['Product'][i] = product

for i in tqdm(range(len(data))):
    if pd.isna(data['Issue Type'][i]):
        input_text = data['structured_conversations'][i] + "\nProducts extracted: " + data['Product'][i]
        issue_type = extract(input_text, ISSUE_TYPE_PROMPT)
        data['Issue Type'][i] = issue_type

for i in tqdm(range(len(data))):
    if pd.isna(data['Services'][i]):
        input_text= data['structured_conversations'][i] + "\n Products extracted: " + data['Product'][i] + "\n Issue types extracted: " + data['Issue Type'][i]
        services = extract(input_text, SERVICES_PROMPT)
        data['Services'][i] = services        

100%|██████████| 11/11 [00:06<00:00,  1.75it/s]


In [20]:
def safe_json_load(value):
    if pd.isna(value):  # catches NaN, None, etc.
        return {}
    if isinstance(value, str) and value.strip():
        try:
            return json.loads(value)
        except json.JSONDecodeError:
            return {}
    elif isinstance(value, dict):
        return value
    return {}

def process_dataframe(df):
    processed_data = []

    for _, row in df.iterrows():
        product_data = safe_json_load(row.get("Product", ""))
        service_data = safe_json_load(row.get("Services", ""))
        issue_data = safe_json_load(row.get("Issue Type", ""))

        entities = {
            "products": product_data.get("product", []) or [],
            "services": service_data.get("service", []) or [],
            "issue_types": issue_data.get("issue_type", []) or []
        }

        # ✨ Replace any np.nan with None (JSON safe)
        entities_clean = json.loads(json.dumps(entities, allow_nan=False))

        processed_data.append({
            "entities": json.dumps(entities_clean)
        })

    return pd.DataFrame(processed_data)


In [21]:
# Extract entities from the DataFrame
processed_df = process_dataframe(data)

In [22]:
# Merge the processed DataFrame with the original DataFrame
data = pd.concat([data, processed_df], axis=1)

In [23]:
data

,user_id,conversations,company_name,cleaned_conversations,structured_conversations,Product,Issue Type,Services,entities
0,182072,Customer: @AmericanAir what happens when a pas...,AmericanAir,Customer what happens when a passenger has pai...,"[{'Company_name': 'AmericanAir'}, {'conversati...","{""product"":[""first class""]}","{""issue_type"":[""broken seat"",""bad service""]}","{""service"":[""AmericanAir customer support"",""sw...","{""products"": [""first class""], ""services"": [""Am..."
1,182072,Customer: @AmericanAir my fingers are crossed ...,AmericanAir,Customer my fingers are crossed I dont miss my...,"[{'Company_name': 'AmericanAir'}, {'conversati...","{""product"":[""connecting flight"",""flights""]}","{""issue_type"":[""inconveniences during vacation""]}","{""service"":[]}","{""products"": [""connecting flight"", ""flights""],..."
2,449215,Customer: Shout-out to @Delta when you find ou...,Delta,Customer Shoutout to when you find out all the...,"[{'Company_name': 'Delta'}, {'conversation': [...","{""product"":[""Delta points"",""platinum"",""rewards...","{""issue_type"":[""points not applied"",""service d...","{""service"":[""Delta Platinum tier"",""Delta rewar...","{""products"": [""Delta points"", ""platinum"", ""rew..."
3,349732,Customer: @GloCare glo please can you helpme r...,GloCare,Customer glo please can you helpme remove the ...,"[{'Company_name': 'GloCare'}, {'conversation':...","{""product"":[""12702friends number"",""Remove frie...","{""issue_type"":[""remove number issue"",""data sha...","{""service"":[""data-sharing""]}","{""products"": [""12702friends number"", ""Remove f..."
4,276962,Customer: @AskPayPal I called back to be told ...,AskPayPal,Customer I called back to be told my call will...,"[{'Company_name': 'AskPayPal'}, {'conversation...","{""product"":[""PayPal"",""eBay""]}","{""issue_type"":[""long wait time"",""call not retu...","{""service"":[""PayPal email"",""DM support""]}","{""products"": [""PayPal"", ""eBay""], ""services"": [..."
5,605003,Customer: @124462 my site is down and I can't ...,mediatemplehelp,Customer my site is down and I cant access ser...,"[{'Company_name': 'mediatemplehelp'}, {'conver...","{""product"":[]}","{""issue_type"":[""site down"",""access server issu...","{""service"":[""support"",""24/7 support""]}","{""products"": [], ""services"": [""support"", ""24/7..."
6,663048,"Customer: @AppleSupport bonsoir, je ne peut pl...",AppleSupport,Customer bonsoir je ne peut plus faire de capt...,"[{'Company_name': 'AppleSupport'}, {'conversat...","{""product"":[""IPhone""]}","{""issue_type"":[""cannot take screenshot""]}","{""service"":[""AppleSupport support via Twitter""]}","{""products"": [""IPhone""], ""services"": [""AppleSu..."
7,535403,Customer: @AskPayPal I’m trying to withdraw fu...,AskPayPal,Customer Im trying to withdraw funds from my a...,"[{'Company_name': 'AskPayPal'}, {'conversation...","{""product"":[""PayPal account""]}","{""issue_type"":[""withdrawal issue"",""incorrect v...","{""service"":[""24/7 chat""]}","{""products"": [""PayPal account""], ""services"": [..."
8,604690,Customer: @AppleSupport are you trying to pull...,AppleSupport,Customer are you trying to pull my pisser Im c...,"[{'Company_name': 'AppleSupport'}, {'conversat...","{""product"":[""iPhone"",""new update""]}","{""issue_type"":[""battery life issue""]}","{""service"":[""AppleSupport customer support"",""D...","{""products"": [""iPhone"", ""new update""], ""servic..."
9,466304,Customer: Freaking #comcast is still at my hou...,comcastcares,Customer Freaking comcast is still at my house...,"[{'Company_name': 'comcastcares'}, {'conversat...","{""product"":[]}","{""issue_type"":[""late service"",""poor communicat...","{""service"":[]}","{""products"": [], ""services"": [], ""issue_types""..."


In [24]:
RELATIONSHIP_PROMPT = """
##################################################################
#  TripleMaker-GPT — single-shot RDF triple extractor            #
##################################################################
You will receive ONE composite JSON input containing:

{
  "Conversation": [                     // array of role+message
    {"role":"Customer","message":"..."},
    {"role":"Company", "message":"..."}
  ],
  "Entities": {                         // from upstream pipelines
    "products":   [ ... ],
    "services":   [ ... ],
    "issue_types":[ ... ]
  }
}

──────────────────────────────────────────────────────────────────
PREDICATES
  hasIssue       product|service  →  issue_type
  providedBy     product|service  →  Company_name
  resolvesWith   issue_type       →  short action

──────────────────────────────────────────────────────────────────
RULES
R0  Fallback service  
    • If products+services are empty but an issue exists, create the
      synthetic service: "<Company_name> service".

R1  Ownership filter for providedBy  
    • Skip providedBy when subject is a known third-party platform:
      Roku, Fire TV, Apple TV, Chromecast, Alexa  (case-insensitive).

R2  Loyalty/tier promotion  
    • For generic tier words (gold, platinum, first class, premier,
      elite, plus) build an alias:
        "<Company_name> <tier> tier"
      and use that alias as a service entity.

R3  hasIssue guard  
    • Do NOT output hasIssue if the object equals "no explicit issue".

R4  resolvesWith extraction  
    • Scan only COMPANY messages.  
    • Action phrase: imperative verb allowed, ≤ 6 words, all lowercase
      (e.g., "restart modem", "send dm", "scan coupon").  
    • Deduplicate identical (subject, action) pairs.

R5  Verbatim entities  
    • Use entities exactly as given (except tier alias & fallback service).  
    • Never invent new entities.

R6  JSON validity  
    • Output a single valid JSON array of triples.  
    • If nothing qualifies, output [].

──────────────────────────────────────────────────────────────────
OUTPUT FORMAT
[
  {"subject":"<entity>","predicate":"<predicate>","object":"<entity|phrase>"},
  ...
]

──────────────────────────────────────────────────────────────────
EXAMPLE
INPUT
{
  "Conversation":[
    {"role":"Customer","message":"My internet plan keeps dropping."},
    {"role":"Company", "message":"Please restart your modem. If that fails, send us a DM."}
  ],
  "Entities":{
    "products":   ["internet plan"],
    "services":   [],
    "issue_types":["frequent disconnections"],
    "Company_name":"Ask_Spectrum"
  }
}

EXPECTED OUTPUT
[
  {"subject":"internet plan","predicate":"hasIssue","object":"frequent disconnections"},
  {"subject":"internet plan","predicate":"providedBy","object":"Ask_Spectrum"},
  {"subject":"frequent disconnections","predicate":"resolvesWith","object":"restart modem"},
  {"subject":"frequent disconnections","predicate":"resolvesWith","object":"send dm"}
]
##################################################################
"""

In [25]:
data['relationship'] = np.nan

In [26]:
for i in range(len(data)):
    if pd.isna(data['relationship'][i]):
        input_text= data['structured_conversations'][i] + "\n Products extracted: " + data['Product'][i] + "\n Issue types extracted: " + data['Issue Type'][i] + "\n Services extracted: " + data['Services'][i]
        services = extract(input_text, RELATIONSHIP_PROMPT)
        data['relationship'][i] = services     

        print("--------------------------------------------------")
        print(input_text)
        print(services)
        print("--------------------------------------------------")

--------------------------------------------------
[{'Company_name': 'AmericanAir'}, {'conversation': [{'role': 'Customer', 'message': 'what happens when a passenger has paid for first class gets a broken inoperable seat Kinda defeats the purpose disapointed'}, {'role': 'Company', 'message': 'Sorry you werent comfy on board David Please check with a crewmember about switching seats'}, {'role': 'Customer', 'message': 'Its a full flight from Hawaii And a night flight so Im not able to sleep'}, {'role': 'Company', 'message': 'Were very sorry for the inconvenience We always want you to have an excellent experience with usCustomer Its been awhile since Ive been happy on an American flight In general bad service on multiple legs of this journey'}]}]
 Products extracted: {"product":["first class"]}
 Issue types extracted: {"issue_type":["broken seat","bad service"]}
 Services extracted: {"service":["AmericanAir customer support","switching seats","excellent experience"]}
[
  {"subject":"first